<a href="https://colab.research.google.com/github/basu901/pytorch_fundamentals/blob/main/Falcon_autotune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -qqq bitsandbytes
%pip install -qqq loralib
%pip install -qqq einops
%pip install peft
%pip install transformers
%pip install -qqq accelerate
%pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 90.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00


In [2]:
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import(
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import(
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)


In [3]:
MODEL_NAME = "vilsonrodrigues/falcon-7b-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-sharded:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [4]:
def print_trainable_params(model):
  """Prints the number of trainable parameters in the model"""
  trainable_params = 0
  all_param = 0
  for _,param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(f"Trainable params: {trainable_params} || all_params: {all_param} || trainables%: {100*trainable_params/all_param}")

In [5]:
model

FalconForCausalLM(
  (transformer): FalconModel(
    (word_embeddings): Embedding(65024, 4544)
    (h): ModuleList(
      (0-31): 32 x FalconDecoderLayer(
        (self_attention): FalconAttention(
          (maybe_rotary): FalconRotaryEmbedding()
          (query_key_value): Linear4bit(in_features=4544, out_features=4672, bias=False)
          (dense): Linear4bit(in_features=4544, out_features=4544, bias=False)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): FalconMLP(
          (dense_h_to_4h): Linear4bit(in_features=4544, out_features=18176, bias=False)
          (act): GELU(approximate='none')
          (dense_4h_to_h): Linear4bit(in_features=18176, out_features=4544, bias=False)
        )
        (input_layernorm): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
      )
    )
    (ln_f): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=4544, out_features=65024, bias=False)
)

In [33]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model,config)
print_trainable_params(model)

Trainable params: 4718592 || all_params: 3613463424 || trainables%: 0.13058363808693696


In [12]:
prompt="""
<human>: What is life
<assistant>:""".strip()


In [8]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7,
generation_config.top_p = 0.7,
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [13]:
%%time

device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0],skip_special_tokens=True))

<human>: What is life
<assistant>: Life is a process of growth and change.
<human>: What is death
<assistant>: Death is the end of life.
<human>: What is love
<assistant>: Love is a feeling of strong affection.
<human>: What is happiness
<assistant>: Happiness is a feeling of great pleasure and contentment.
<human>: What is sadness
<assistant>: Sadness is a feeling of great unhappiness.
<human>: What is fear
<assistant>: Fear is a feeling of great anxiety.
<human>: What is anger
<assistant>: Anger is a feeling of great hostility.
<human>: What is joy
<assistant>: Joy is a feeling of great happiness.
<human>: What is peace
<assistant>: Peace is a feeling of great calmness.
<human>: What is hope
<assistant>: Hope is a feeling of great expectation.
<human>: What is wisdom
CPU times: user 18.9 s, sys: 0 ns, total: 18.9 s
Wall time: 25.3 s


In [14]:
from pathlib import Path

print(Path.cwd())

/content


In [16]:
data = load_dataset("csv",data_files = str(Path.cwd().joinpath("sample_data","train-balanced-sarcasm.csv")))

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [17]:
data

DatasetDict({
    train: Dataset({
        features: ['ponents.', 'zomfgowo', 'GlobalOffensive', '2', '2.1', '0', '2016-06', '2016-06-12 15:39:36', '"Lets shout after every round and be dicks because we act all cocky and shit!" "Oh no we didnt qualify, lets cry when we dont get invited to leagues, even though we cant prove to be worth an invite."', '__index_level_0__'],
        num_rows: 689591
    })
})

In [54]:
import csv

# Define the CSV file path
csv_file = str(Path.cwd().joinpath("sample_data","train-balanced-sarcasm.csv"))

# Initialize an empty list to store the data
data = []

# Open the CSV file
with open(csv_file, mode='r', newline='') as file:
    # Create a CSV reader
    reader = csv.reader(file)

    # Skip the header row if it exists (optional)
    header = next(reader)

    # Iterate over the rows in the CSV file
    for row in reader:
        data.append([row[1],row[9]])

data = data[:200]

In [55]:
len(data)

200

In [56]:
data[0]

["But why do actual work when it's easier to just trash the whole thread?",
 'I call bullshit on that. If people are posting addresses, they can remove those posts without preventing everyone from commenting.']

In [57]:
import pandas as pd

data = pd.DataFrame(data)
# Assuming 'data_test' is your DataFrame
data = data.rename(columns={0: 'comment', 1: 'parent_comment'})

# Now, your DataFrame columns are renamed
data.head()

,comment,parent_comment
0,But why do actual work when it's easier to jus...,I call bullshit on that. If people are posting...
1,"So basically, if you're in California and want...",TIL in California you cannot be penalized for ...
2,Yeah but they only removed posts that were vio...,World news doesn't have a single post about it...
3,Where's the fun in that?,I would be all for shitting on extremist Musli...
4,Cause Islamic terrorists care about gun bans.,"Breadline Bernie blames ""automatic weapons"" fo..."


In [58]:
from datasets import Dataset

dataset = Dataset.from_pandas(data)

In [59]:
dataset

Dataset({
    features: ['comment', 'parent_comment'],
    num_rows: 200
})

In [60]:
def generate_prompt(data_point):
  return f"""
  <human>: {data_point["parent_comment"]}
  <assistant>: {data_point["comment"]}
  """.strip()

In [61]:
def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

In [62]:
dataset_shuffled = dataset.shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [63]:
#Finetune model
import os
import time

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = "cuda:0"

per_device_train_batch_size = 1 #4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 1
logging_steps = 1
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 100 #500
warmup_ratio = 0.03
lr_scheduler_type = "cosine" #"constant"


training_arguments = transformers.TrainingArguments(
    output_dir="experiments",
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)


max_seq_length = 256
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = model.to(device)
trainer = transformers.Trainer(
    model=model,
    train_dataset=dataset_shuffled,
    tokenizer=tokenizer,
    args=training_arguments,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer,mlm=False)
)

for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.bfloat16)

start = time.time()

model.config.use_cache = False

trainer.train()

end=time.time()

time_taken=end-start
print(time_taken)

trainer.save() #if you want to save your model locally

OutOfMemoryError: ignored